## **Load Libraires**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

In [2]:
warnings.filterwarnings('ignore')

# **Feature Engineering**

In [3]:
df = pd.read_csv('C:/Users/amado/Desktop/Vehicules/data/cars.csv')

In [4]:
class Intervals():
    
    def __init__(self,feature):
        
        self.mean=np.mean(feature)
        self.sd=np.std(feature)
        self.interval_range=[1.5,2,2.5,3.0,3.5,4]
        
    def Upper_Interval(self):
        
        for interval in self.interval_range:
            
            upper_interval= self.mean+interval*self.sd
            upper_interval=np.round(upper_interval)
            
            print(f"Interval range STD {interval}: {upper_interval}")
        
    def Lower_Interval(self):
        
         for interval in self.interval_range:
            
            lower_interval=self.mean-interval*self.sd
            lower_interval=np.round(lower_interval)
            
            print(f"Interval range STD {interval}: {lower_interval}")

In [5]:
class IntervalFit(Intervals):
    def __init__(self,feature):
        Intervals.__init__(self,feature)

In [6]:
df.price.describe()

count     64132.000000
mean      18404.653543
std       10855.717134
min         495.000000
25%       10930.000000
50%       15999.000000
75%       22980.000000
max      159999.000000
Name: price, dtype: float64

In [7]:
df = df.query("price > 1000")

In [8]:
df.mileage.describe()

count     64116.000000
mean      23460.128439
std       21407.528573
min           1.000000
25%        7531.750000
50%       17766.000000
75%       32971.000000
max      323000.000000
Name: mileage, dtype: float64

It is very rare that a vehicle only has one mile traveled, so it is an outlier.

In [9]:
IntervalFit(df.mileage).Upper_Interval()

Interval range STD 1.5: 55571.0
Interval range STD 2: 66275.0
Interval range STD 2.5: 76979.0
Interval range STD 3.0: 87682.0
Interval range STD 3.5: 98386.0
Interval range STD 4: 109090.0


The range with a 2.5 SSD is a reasonable upper range.

### *Mileage*

In [10]:
df = df.query("mileage > 3106 and mileage < 76749")

### *Year*

In [11]:
df.year.describe()

count    55119.000000
mean      2016.916762
std          1.902691
min       1970.000000
25%       2016.000000
50%       2017.000000
75%       2018.000000
max       2060.000000
Name: year, dtype: float64

It is impossible that there is a manufacturing date of the year 2060. It is a clear data collection failure.

In [12]:
df = df.query('year >= 2000 and year <= 2020')

### *MPG*

In [13]:
df.mpg.describe()

count    55104.000000
mean        57.139837
std         17.932148
min          1.100000
25%         47.900000
50%         56.500000
75%         65.700000
max        470.800000
Name: mpg, dtype: float64

Generally, most vehicles have an MPG greater than 20. Therefore, we will select all vehicles that meet this criteria.

In [14]:
df = df.query("mpg > 20")

## *Price*

Because it is the predictor variable, we must be very careful when eliminating the data. For which I will obtain to create subsets according to the manufacturer.

### *Ford*

In [15]:
ford = df.query("manufacturer == 'Ford'")

In [16]:
ford.price.describe()

count    16436.000000
mean     12090.512473
std       4347.430964
min       1294.000000
25%       8999.000000
50%      11000.000000
75%      14986.000000
max      54995.000000
Name: price, dtype: float64

In [17]:
ford.query('price >  40000')

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,manufacturer
302,Mustang,2020,42489,Automatic,3500,Petrol,145,22.1,5.0,Ford
7164,Mustang,2020,40495,Semi-Auto,3200,Petrol,145,24.8,5.0,Ford
11910,Mustang,2017,48999,Automatic,7546,Petrol,145,23.5,5.0,Ford
11912,Focus,2018,54995,Manual,11000,Petrol,145,36.7,2.3,Ford
11913,Mustang,2017,49999,Manual,21575,Petrol,145,20.9,5.0,Ford


* I do not find a logical explanation that supports the high price of the *Ford Focus*. Since it is a mid-range vehicle.



* Model year 2017 *Ford Mustang* models are priced higher than a more recent model. In addition to having a higher mileage.

In [18]:
ford = ford.query('price < 43000')

Select those vehicles with an amount less than 43k £.

In [19]:
ford.query('price >  40000')

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,manufacturer
302,Mustang,2020,42489,Automatic,3500,Petrol,145,22.1,5.0,Ford
7164,Mustang,2020,40495,Semi-Auto,3200,Petrol,145,24.8,5.0,Ford


## *Toyota*

In [20]:
toyota = df.query("manufacturer == 'Toyota'")

In [21]:
toyota.price.describe()

count     6089.000000
mean     12300.046313
std       5847.002112
min       1795.000000
25%       8102.000000
50%      10695.000000
75%      14871.000000
max      59995.000000
Name: price, dtype: float64

In [22]:
toyota.query('price > 42000')

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,manufacturer
59191,Land Cruiser,2019,42444,Semi-Auto,10083,Diesel,145,30.1,2.8,Toyota
59193,Land Cruiser,2019,42995,Semi-Auto,16634,Diesel,145,30.1,2.8,Toyota
59195,Land Cruiser,2020,50995,Semi-Auto,3390,Diesel,145,30.1,2.8,Toyota
59197,Land Cruiser,2019,45995,Semi-Auto,8967,Diesel,150,30.1,2.8,Toyota
59201,Land Cruiser,2019,50995,Semi-Auto,6254,Diesel,145,30.1,2.8,Toyota
59205,Land Cruiser,2019,47795,Semi-Auto,8813,Diesel,145,30.1,2.8,Toyota
59209,Land Cruiser,2019,44935,Semi-Auto,15200,Diesel,145,30.1,2.8,Toyota
59211,Land Cruiser,2019,42990,Semi-Auto,22845,Diesel,150,30.1,2.8,Toyota
59216,Land Cruiser,2020,50995,Automatic,3500,Diesel,145,30.1,2.8,Toyota
59223,Land Cruiser,2019,44995,Automatic,4512,Diesel,145,30.1,2.8,Toyota


Vehicles over £40k are heavily dominated by the Land Cruiser model.

In [23]:
toyota.query("model == ' Supra'")

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,manufacturer
59241,Supra,2019,51995,Semi-Auto,4153,Petrol,145,34.5,3.0,Toyota
59245,Supra,2019,59995,Automatic,9909,Other,150,34.5,3.0,Toyota


There is only one single value. Which is better to remove as it can skew the model.

In [24]:
toyota = toyota.query("model != ' Supra'")

In [25]:
land_cruiser = toyota.query("model == ' Land Cruiser'")

In [26]:
other_vehicles_toyota = toyota.query("model != ' Land Cruiser'")

In [27]:
land_cruiser_2019 = land_cruiser.query("year == 2019")
land_cruiser_2020 = land_cruiser.query("year == 2020")

In [28]:
land_cruiser = land_cruiser.query("year != 2019 and year != 2020")

In [29]:
land_cruiser_2020

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,manufacturer
59195,Land Cruiser,2020,50995,Semi-Auto,3390,Diesel,145,30.1,2.8,Toyota
59216,Land Cruiser,2020,50995,Automatic,3500,Diesel,145,30.1,2.8,Toyota
59229,Land Cruiser,2020,54550,Automatic,4000,Diesel,150,30.1,2.8,Toyota


For the 2020 Toyota Land Cruiser. It has a higher price and with a considerable amount of mileage.

In [30]:
land_cruiser_2020 = land_cruiser_2020.query("price < 51000")

In [31]:
land_cruiser_2020

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,manufacturer
59195,Land Cruiser,2020,50995,Semi-Auto,3390,Diesel,145,30.1,2.8,Toyota
59216,Land Cruiser,2020,50995,Automatic,3500,Diesel,145,30.1,2.8,Toyota


In [32]:
land_cruiser_2019

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,manufacturer
59183,Land Cruiser,2019,39498,Semi-Auto,11712,Diesel,145,30.1,2.8,Toyota
59187,Land Cruiser,2019,39498,Semi-Auto,12543,Diesel,145,30.1,2.8,Toyota
59191,Land Cruiser,2019,42444,Semi-Auto,10083,Diesel,145,30.1,2.8,Toyota
59192,Land Cruiser,2019,40999,Semi-Auto,11619,Diesel,145,30.1,2.8,Toyota
59193,Land Cruiser,2019,42995,Semi-Auto,16634,Diesel,145,30.1,2.8,Toyota
59197,Land Cruiser,2019,45995,Semi-Auto,8967,Diesel,150,30.1,2.8,Toyota
59200,Land Cruiser,2019,40995,Semi-Auto,11404,Diesel,145,30.1,2.8,Toyota
59201,Land Cruiser,2019,50995,Semi-Auto,6254,Diesel,145,30.1,2.8,Toyota
59205,Land Cruiser,2019,47795,Semi-Auto,8813,Diesel,145,30.1,2.8,Toyota
59209,Land Cruiser,2019,44935,Semi-Auto,15200,Diesel,145,30.1,2.8,Toyota


For the same vehicle with a slightly older model it has a higher price. Therefore, we will proceed to eliminate them.

In [33]:
land_cruiser_2019 = land_cruiser_2019.query("price < 48000")

In [34]:
land_cruiser_2019

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,manufacturer
59183,Land Cruiser,2019,39498,Semi-Auto,11712,Diesel,145,30.1,2.8,Toyota
59187,Land Cruiser,2019,39498,Semi-Auto,12543,Diesel,145,30.1,2.8,Toyota
59191,Land Cruiser,2019,42444,Semi-Auto,10083,Diesel,145,30.1,2.8,Toyota
59192,Land Cruiser,2019,40999,Semi-Auto,11619,Diesel,145,30.1,2.8,Toyota
59193,Land Cruiser,2019,42995,Semi-Auto,16634,Diesel,145,30.1,2.8,Toyota
59197,Land Cruiser,2019,45995,Semi-Auto,8967,Diesel,150,30.1,2.8,Toyota
59200,Land Cruiser,2019,40995,Semi-Auto,11404,Diesel,145,30.1,2.8,Toyota
59205,Land Cruiser,2019,47795,Semi-Auto,8813,Diesel,145,30.1,2.8,Toyota
59209,Land Cruiser,2019,44935,Semi-Auto,15200,Diesel,145,30.1,2.8,Toyota
59211,Land Cruiser,2019,42990,Semi-Auto,22845,Diesel,150,30.1,2.8,Toyota


In [35]:
land_cruiser = pd.concat([land_cruiser,land_cruiser_2019,land_cruiser_2020])

In [36]:
toyota = pd.concat([other_vehicles_toyota,land_cruiser])

In [37]:
toyota.query("price > 42000")

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,manufacturer
59232,Land Cruiser,2014,44990,Automatic,60000,Diesel,540,29.7,4.5,Toyota
59191,Land Cruiser,2019,42444,Semi-Auto,10083,Diesel,145,30.1,2.8,Toyota
59193,Land Cruiser,2019,42995,Semi-Auto,16634,Diesel,145,30.1,2.8,Toyota
59197,Land Cruiser,2019,45995,Semi-Auto,8967,Diesel,150,30.1,2.8,Toyota
59205,Land Cruiser,2019,47795,Semi-Auto,8813,Diesel,145,30.1,2.8,Toyota
59209,Land Cruiser,2019,44935,Semi-Auto,15200,Diesel,145,30.1,2.8,Toyota
59211,Land Cruiser,2019,42990,Semi-Auto,22845,Diesel,150,30.1,2.8,Toyota
59223,Land Cruiser,2019,44995,Automatic,4512,Diesel,145,30.1,2.8,Toyota
59195,Land Cruiser,2020,50995,Semi-Auto,3390,Diesel,145,30.1,2.8,Toyota
59216,Land Cruiser,2020,50995,Automatic,3500,Diesel,145,30.1,2.8,Toyota


A 2014 model is priced similarly to a 2019. We are going to proceed to remove it. It also has considerably better mileage.

In [38]:
toyota = toyota.query("price != 44990")

In [39]:
toyota.query("price > 42000")

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,manufacturer
59191,Land Cruiser,2019,42444,Semi-Auto,10083,Diesel,145,30.1,2.8,Toyota
59193,Land Cruiser,2019,42995,Semi-Auto,16634,Diesel,145,30.1,2.8,Toyota
59197,Land Cruiser,2019,45995,Semi-Auto,8967,Diesel,150,30.1,2.8,Toyota
59205,Land Cruiser,2019,47795,Semi-Auto,8813,Diesel,145,30.1,2.8,Toyota
59209,Land Cruiser,2019,44935,Semi-Auto,15200,Diesel,145,30.1,2.8,Toyota
59211,Land Cruiser,2019,42990,Semi-Auto,22845,Diesel,150,30.1,2.8,Toyota
59223,Land Cruiser,2019,44995,Automatic,4512,Diesel,145,30.1,2.8,Toyota
59195,Land Cruiser,2020,50995,Semi-Auto,3390,Diesel,145,30.1,2.8,Toyota
59216,Land Cruiser,2020,50995,Automatic,3500,Diesel,145,30.1,2.8,Toyota


## *Hyundi*

In [40]:
hyundi = df.query("manufacturer == 'Hyundi'")

In [41]:
hyundi.price.describe()

count     4248.000000
mean     12111.739642
std       5380.656208
min       1295.000000
25%       7995.000000
50%      11362.500000
75%      15199.250000
max      92000.000000
Name: price, dtype: float64

In [42]:
hyundi.query("price > 40000")

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,manufacturer
63519,I10,2017,92000,Automatic,35460,Petrol,150,47.9,1.2,Hyundi


It is a clear outlier. Since it is a relatively economical vehicle.

In [43]:
IntervalFit(hyundi.price).Upper_Interval()

Interval range STD 1.5: 20182.0
Interval range STD 2: 22872.0
Interval range STD 2.5: 25562.0
Interval range STD 3.0: 28252.0
Interval range STD 3.5: 30942.0
Interval range STD 4: 33632.0


In [44]:
hyundi.query("price > 33000")

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,manufacturer
59442,Santa Fe,2019,34998,Semi-Auto,4133,Diesel,145,39.2,2.2,Hyundi
59443,Santa Fe,2019,34998,Semi-Auto,4956,Diesel,145,39.2,2.2,Hyundi
59766,Santa Fe,2019,33998,Semi-Auto,3122,Diesel,145,39.8,2.2,Hyundi
60268,Santa Fe,2019,34000,Semi-Auto,5931,Diesel,145,39.2,2.2,Hyundi
60358,Santa Fe,2019,33995,Automatic,6363,Diesel,145,42.8,2.2,Hyundi
60884,Santa Fe,2019,36495,Automatic,5000,Diesel,145,39.8,2.2,Hyundi
61053,Santa Fe,2020,36495,Semi-Auto,6073,Diesel,145,39.2,2.2,Hyundi
61158,Santa Fe,2019,35490,Semi-Auto,17568,Diesel,150,39.8,2.2,Hyundi
61373,Santa Fe,2019,33970,Semi-Auto,3580,Diesel,145,39.8,2.2,Hyundi
61559,Santa Fe,2019,33495,Semi-Auto,6209,Diesel,145,39.2,2.2,Hyundi


The Hyundai Santa Fe strongly dominates this price range.

In [45]:
hyundi = hyundi.query("price < 37000")

### *Audi*

In [46]:
audi = df.query("manufacturer == 'Audi'")

In [47]:
audi.price.describe()

count      8931.000000
mean      22126.330646
std       10688.013208
min        3990.000000
25%       15000.000000
50%       19655.000000
75%       26162.500000
max      129000.000000
Name: price, dtype: float64

In [48]:
IntervalFit(audi.price).Lower_Interval()

Interval range STD 1.5: 6095.0
Interval range STD 2: 752.0
Interval range STD 2.5: -4592.0
Interval range STD 3.0: -9936.0
Interval range STD 3.5: -15280.0
Interval range STD 4: -20623.0


In [49]:
audi  = audi.query("price > 6000")

In [50]:
audi.query("price > 100000")

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,manufacturer
19612,R8,2019,129000,Semi-Auto,4000,Petrol,145,21.4,5.2,Audi
21905,R8,2019,112990,Automatic,8175,Petrol,145,21.6,5.2,Audi
22708,R8,2019,117990,Automatic,11936,Petrol,145,21.4,5.2,Audi
22920,R8,2019,109990,Automatic,6954,Petrol,145,21.6,5.2,Audi
25411,R8,2019,119995,Semi-Auto,7800,Petrol,145,21.4,5.2,Audi
28434,R8,2019,125000,Automatic,13663,Petrol,150,21.4,5.2,Audi


There is a higher priced 2019 model year R8 that has more mileage. So I will choose to remove it.

In [51]:
audi_R8 = audi.query("model == ' R8'")
audi_R8_2019 = audi_R8.query("year == 2019")
audi_R8 = audi_R8.query("year != 2019")

In [52]:
other_audi_cars = audi.query("model != ' R8'")

In [53]:
audi_R8_2019

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,manufacturer
19612,R8,2019,129000,Semi-Auto,4000,Petrol,145,21.4,5.2,Audi
21905,R8,2019,112990,Automatic,8175,Petrol,145,21.6,5.2,Audi
22708,R8,2019,117990,Automatic,11936,Petrol,145,21.4,5.2,Audi
22920,R8,2019,109990,Automatic,6954,Petrol,145,21.6,5.2,Audi
25411,R8,2019,119995,Semi-Auto,7800,Petrol,145,21.4,5.2,Audi
28434,R8,2019,125000,Automatic,13663,Petrol,150,21.4,5.2,Audi


In [54]:
audi_R8_2019 = audi_R8_2019.query("price != 125000 and price != 117990")

In [55]:
audi_R8_2019

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,manufacturer
19612,R8,2019,129000,Semi-Auto,4000,Petrol,145,21.4,5.2,Audi
21905,R8,2019,112990,Automatic,8175,Petrol,145,21.6,5.2,Audi
22920,R8,2019,109990,Automatic,6954,Petrol,145,21.6,5.2,Audi
25411,R8,2019,119995,Semi-Auto,7800,Petrol,145,21.4,5.2,Audi


In [56]:
R8 = pd.concat([audi_R8,audi_R8_2019])

In [57]:
audi = pd.concat([other_audi_cars,R8])

In [58]:
audi.query("price > 100000")

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,manufacturer
19612,R8,2019,129000,Semi-Auto,4000,Petrol,145,21.4,5.2,Audi
21905,R8,2019,112990,Automatic,8175,Petrol,145,21.6,5.2,Audi
22920,R8,2019,109990,Automatic,6954,Petrol,145,21.6,5.2,Audi
25411,R8,2019,119995,Semi-Auto,7800,Petrol,145,21.4,5.2,Audi


## *BMW*

In [59]:
bmw = df.query("manufacturer == 'BMW'")

In [60]:
bmw.price.describe()

count      8528.000000
mean      21550.004104
std       10332.236635
min        2795.000000
25%       14743.750000
50%       19064.500000
75%       25733.750000
max      123456.000000
Name: price, dtype: float64

In [61]:
IntervalFit(bmw.price).Lower_Interval()

Interval range STD 1.5: 6053.0
Interval range STD 2: 887.0
Interval range STD 2.5: -4279.0
Interval range STD 3.0: -9445.0
Interval range STD 3.5: -14611.0
Interval range STD 4: -19777.0


In [62]:
bmw.query("price < 3000")

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,manufacturer
51772,3 Series,2006,2795,Manual,76000,Diesel,160,50.4,2.0,BMW


In [63]:
bmw.query("price > 75000")

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,manufacturer
41994,X7,2020,78000,Semi-Auto,5000,Diesel,150,31.4,3.0,BMW
42474,8 Series,2019,84898,Semi-Auto,3185,Petrol,145,24.4,4.4,BMW
44295,X7,2020,77990,Semi-Auto,5656,Diesel,150,31.4,3.0,BMW
44297,X7,2019,77880,Semi-Auto,6506,Diesel,150,31.4,3.0,BMW
44355,X7,2020,78490,Semi-Auto,4919,Diesel,145,31.4,3.0,BMW
44691,X7,2019,77880,Semi-Auto,6506,Diesel,145,31.4,3.0,BMW
45391,2 Series,2015,123456,Semi-Auto,33419,Diesel,20,68.9,2.0,BMW
48865,X7,2020,77995,Semi-Auto,7500,Diesel,150,31.4,3.0,BMW
49147,M4,2016,76990,Automatic,4550,Petrol,300,33.2,3.0,BMW


The 2 series model I do not find an explanation that supports the high price. Since it exceeds the price of the most expensive vehicles of the brand. It is also a model with more mileage and a longer manufacturing date.

In [64]:
bmw = bmw.query("price > 6000 and price < 85000")

In [65]:
bmw.query("price > 75000")

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,manufacturer
41994,X7,2020,78000,Semi-Auto,5000,Diesel,150,31.4,3.0,BMW
42474,8 Series,2019,84898,Semi-Auto,3185,Petrol,145,24.4,4.4,BMW
44295,X7,2020,77990,Semi-Auto,5656,Diesel,150,31.4,3.0,BMW
44297,X7,2019,77880,Semi-Auto,6506,Diesel,150,31.4,3.0,BMW
44355,X7,2020,78490,Semi-Auto,4919,Diesel,145,31.4,3.0,BMW
44691,X7,2019,77880,Semi-Auto,6506,Diesel,145,31.4,3.0,BMW
48865,X7,2020,77995,Semi-Auto,7500,Diesel,150,31.4,3.0,BMW
49147,M4,2016,76990,Automatic,4550,Petrol,300,33.2,3.0,BMW


## *Mercedes*

In [66]:
mercedes = df.query("manufacturer== 'Mercedes'")

In [67]:
mercedes.price.describe()

count     10848.000000
mean      23196.512076
std        9743.955800
min        1495.000000
25%       16991.000000
50%       20990.000000
75%       26995.000000
max      139995.000000
Name: price, dtype: float64

In [68]:
mercedes = mercedes.query("price > 6000")

In [69]:
mercedes.price.describe()

count     10801.000000
mean      23277.442089
std        9687.170004
min        6495.000000
25%       16998.000000
50%       20995.000000
75%       26998.000000
max      139995.000000
Name: price, dtype: float64

## *Unique Values*

In [70]:
df2 = pd.concat([ford,toyota,hyundi,mercedes,bmw,audi])

In [71]:
models = df2.model.value_counts()

In [72]:
models[models < 10]

 Puma               9
 IQ                 8
 SQ7                8
 M6                 7
 S4                 6
 PROACE VERSO       6
 Getz               4
 S8                 4
 Urban Cruiser      4
 Verso-S            3
 Veloster           3
 GLB Class          2
 S5                 2
 Streetka           2
 R Class            2
 Accent             1
 RS7                1
200                 1
 Z3                 1
220                 1
Focus               1
 Transit Tourneo    1
A Class             1
 Amica              1
180                 1
Name: model, dtype: int64

In [73]:
models = models[models < 10]

In [74]:
df2['model'] = df2['model'].apply(lambda x: "Other" if x in models else x)

In [75]:
df2.query("model == 'Other'").head()

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,manufacturer
122,Other,2020,21995,Manual,4111,Petrol,145,50.4,1.0,Ford
1141,Other,2020,20991,Manual,5750,Petrol,145,49.6,1.0,Ford
3318,Other,2020,20491,Manual,4998,Petrol,145,50.4,1.0,Ford
5880,Other,2020,21500,Manual,3198,Petrol,145,50.4,1.0,Ford
7117,Other,2020,20590,Manual,3214,Petrol,145,50.4,1.0,Ford


In [76]:
engineSize = df2.engineSize.value_counts()

In [77]:
engineSize[engineSize < 10]

6.2    9
3.2    8
0.6    7
4.1    2
6.6    2
6.0    1
6.3    1
2.7    1
Name: engineSize, dtype: int64

In [78]:
engineSize = engineSize[engineSize < 10]

In [79]:
df2['engineSize'] = df2['engineSize'].apply(lambda x: "Other" if x in engineSize else x)

In [80]:
df2.engineSize.unique()

array([1.0, 1.5, 1.6, 1.2, 2.0, 1.1, 2.3, 1.4, 5.0, 2.2, 2.5, 1.8, 1.3,
       0.0, 2.4, 3.0, 2.8, 1.7, 2.1, 5.5, 4.0, 2.9, 3.5, 4.7, 'Other',
       4.4, 4.2, 5.2], dtype=object)

In [81]:
df2.transmission.unique()

array(['Automatic', 'Manual', 'Semi-Auto', 'Other'], dtype=object)

### *Delete Other Values*

Since they do not add significant value.

In [82]:
df2 = df2.query("model != 'Other'")
df2 = df2.query("engineSize	!= 'Other'")
df2 = df2.query("transmission != 'Other'")
df2 = df2.query("fuelType != 'Other' and fuelType != 'Electric'")

In [83]:
df2.fuelType.unique()

array(['Petrol', 'Diesel', 'Hybrid'], dtype=object)

In [84]:
df2.engineSize.unique()

array([1.0, 1.5, 1.6, 1.2, 2.0, 1.1, 2.3, 1.4, 5.0, 2.2, 2.5, 1.8, 1.3,
       0.0, 2.4, 3.0, 2.8, 1.7, 2.1, 5.5, 4.0, 2.9, 3.5, 4.7, 4.4, 4.2,
       5.2], dtype=object)

In [85]:
df2.engineSize = df2.engineSize.astype(float)

In [86]:
df2.engineSize.unique()

array([1. , 1.5, 1.6, 1.2, 2. , 1.1, 2.3, 1.4, 5. , 2.2, 2.5, 1.8, 1.3,
       0. , 2.4, 3. , 2.8, 1.7, 2.1, 5.5, 4. , 2.9, 3.5, 4.7, 4.4, 4.2,
       5.2])

In [87]:
df2.query("model == 'Other'")

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,manufacturer


## *Upper Range MPG according to Fuel Type*

In [89]:
df2.mpg.describe()

count    54744.000000
mean        57.075716
std         16.845197
min         20.300000
25%         47.900000
50%         56.500000
75%         65.700000
max        470.800000
Name: mpg, dtype: float64

In [90]:
def UpperLimit(x):
    mean = np.mean(x)
    std = np.std(x)
    
    return mean + 3.5 * std

In [91]:
df2.groupby('fuelType')[['mpg']].apply(UpperLimit).sort_values(by = 'mpg',ascending = False)

,mpg
fuelType,
Hybrid,265.218797
Diesel,96.956139
Petrol,88.109955


In [92]:
df3 = df2.query("mpg < 265")

In [93]:
df3.mpg.describe()

count    54712.000000
mean        56.833737
std         13.555571
min         20.300000
25%         47.900000
50%         56.500000
75%         65.700000
max        256.800000
Name: mpg, dtype: float64

## *Save Data Clear*

In [114]:
df3.to_csv('C:/Users/amado/Desktop/Vehicules/data/dataset_car_clear.csv',index = False)